In [ ]:
# %%
import numpy as np
import librosa
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# speech-silence and voice-unvoiced


def read_lab(lab_file_name: str):
    """Read lab file
    lab_file_name: str, name of lab file
    Return: list of lists [start_time, end_time, label]
    """
    data = []
    with open(lab_file_name) as f:
        for line in f.readlines():
            data.append(line.split())
    return data


def get_closest(arr, values):
    """Get closest value in an sorted array
    arr: np.ndarray
    values: List of values to find
    Return: List of closest values to input values
    """
    arr = np.array(arr)
    values = np.array(values, dtype=np.float64)
    idx = np.searchsorted(arr, values)
    idx = np.array(idx)
    idx[arr[idx] - values > np.diff(arr).mean() * 0.5] -= 1
    return arr[idx]


def get_closest_idx(arr, values):
    """Get closest index in an sorted array
    arr: np.ndarray
    values: List of values to find
    Return: List of closest index to input values
    """
    arr = np.array(arr)
    values = np.array(values, dtype=np.float64)
    idx = np.searchsorted(arr, values, side='left')
    idx = np.array(idx)
    return idx


def array_norm(arr: np.ndarray):
    min_arr=np.min(arr)
    max_arr=np.max(arr)
    return (arr-min_arr)/(max_arr-min_arr)

def array_norm_by_threshold(arr: np.ndarray, threshold):
    """Normalize given function by threshold
    arr: np.ndarray
    T: threshold
    """
    min_arr = min(arr)
    max_arr = max(arr)
    return np.where(arr >= threshold, (arr-threshold)/(max_arr-threshold), (arr-threshold)/(threshold-min_arr))





def load_data(audio_name: str):
    signal, sr = librosa.load(f'{audio_name}.wav')
    lab_data = read_lab(f'{audio_name}.lab')
    mean_std = lab_data[-2:]
    lab_data = lab_data[:-2]
    timestamp_label = list(
        filter(lambda x: (x[2] == 'v' or x[2] == 'uv'), lab_data))
    t_i = 0
    t_f = signal.shape[0] / sr
    t = np.linspace(t_i, t_f, num=signal.shape[0])
    t_without_sl = remove_sl(t, timestamp_label, t)
    signal = remove_sl(signal, timestamp_label, t)
    return signal, sr, t_without_sl, timestamp_label


def separate_frames(signal, sr, t, frame_length=0.02):
    """Separate signal into frames
    signal: np.ndarray
    sr: sampling rate
    t: time array
    frame_length: length of frame
    Return: Array of frames
    """
    frame_size = int(sr * frame_length)
    frame_count = len(signal) // frame_size
    signal_frames = []
    for i in range(0, frame_count * frame_size, frame_size):
        signal_frames.append(signal[i:i + frame_size])
    return np.array(signal_frames), frame_size, frame_count


def calc_STE(signal_frames):
    """Calculate STE
    signal_frames: Array of frames
    Return: Array of STE for each frame
    """
    STE = []
    frame_size = len(signal_frames[0])
    frames_count = len(signal_frames)
    for i in range(frames_count):
        value = np.sum(np.square(signal_frames[i])) / np.ones(frame_size)
        STE.append(value)
    STE = np.array(STE)
    STE = STE.reshape(-1)
    return STE


def calc_ZCR(signal_frames):
    """Calculate ZCR
    signal_frames: Array of frames
    Return: Array of ZCR for each frame
    """
    ZCR = []
    frames_count = len(signal_frames)
    frame_size = len(signal_frames[0])
    for i in range(frames_count):
        value = np.sum(
            np.abs(np.diff(np.sign(signal_frames[i])))) / np.ones(frame_size)
        ZCR.append(value)
    ZCR = np.array(ZCR)
    ZCR = ZCR.reshape(-1)
    return ZCR


def calc_LF(signal_frames, sr):
    """Calculate LF
    signal_frames: Array of frames
    sr: sampling rate
    Return: List of LF for each frame
    """
    filtered_wave=[]
    for frame in signal_frames:
        n=len(frame)
        d=1/sr
        hs=np.fft.rfft(frame)
        fs=np.fft.rfftfreq(n, d)
        hs[abs(fs)>1000]*=0
        ys=np.fft.irfft(hs)
        ys=np.concatenate((ys, np.array(ys[-1]).reshape(-1)))
        filtered_wave.append(ys)
    LF=calc_STE(filtered_wave)/calc_STE(signal_frames)
    return LF


def separate_vu(STE, timestamp_label, t):
    STE_v = np.array([])
    STE_uv = np.array([])
    for line in timestamp_label:
        if line[2] == 'v':
            try:
                idx1 = int(get_closest_idx(t, line[0]))
                idx2 = int(get_closest_idx(t, line[1]))
                STE_v = np.append(STE_v, STE[idx1:idx2])
            except:
                print(line)
        if line[2] == 'uv':
            try:
                idx1 = int(get_closest_idx(t, line[0]))
                idx2 = int(get_closest_idx(t, line[1]))
                STE_uv = np.append(STE_uv, STE[idx1:idx2])
            except:
                print(line)
    return np.array(STE_v), np.array(STE_uv)



